<a href="https://colab.research.google.com/github/Celestia-Analytics/exoplanet-data-visualization/blob/main/exoplanet_trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Hello World!")

Hello World!
